도구 (Tools)

도구(Tool)는 에이전트, 체인 또는 LLM이 외부 세계와 상호작용하기 위한 인터페이스입니다.

LangChain 에서 기본 제공하는 도구를 사용하여 쉽게 도구를 활용할 수 있으며, 사용자 정의 도구(Custom Tool) 를 쉽게 구축하는 것도 가능합니다.

In [10]:
# API 키를 환경변수로 관리하기 위한 설정 파일
from dotenv import load_dotenv

# API 키 정보 로드
load_dotenv()

True

In [1]:
# LangSmith 추적을 설정합니다. https://smith.langchain.com
# !pip install -qU langchain-teddynote
from langchain_teddynote import logging

# 프로젝트 이름을 입력합니다.
logging.langsmith("CH15-Tools")

LangSmith 추적을 시작합니다.
[프로젝트명]
CH15-Tools


In [2]:
import warnings

# 경고 메시지 무시
warnings.filterwarnings("ignore")

In [4]:
!pip install langchain_experimental

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.2/209.2 kB 5.7 MB/s eta 0:00:00

[notice] A new release of pip is available: 24.0 -> 25.0.1
[notice] To update, run: pip install --upgrade pip


빌트인 도구(built-in tools)

랭체인에서 제공하는 사전에 정의된 도구(tool) 와 툴킷(toolkit) 을 사용할 수 있습니다.

tool 은 단일 도구를 의미하며, toolkit 은 여러 도구를 묶어서 하나의 도구로 사용할 수 있습니다.

관련 도구는 아래의 링크에서 참고하실 수 있습니다.

In [5]:
from langchain_experimental.tools import PythonREPLTool

# 파이썬 코드를 실행하는 도구를 생성합니다.
python_tool = PythonREPLTool()

In [6]:
# 파이썬 코드를 실행하고 결과를 반환합니다.
print(python_tool.invoke("print(100 + 200)"))

Python REPL can execute arbitrary code. Use with caution.


300



아래는 LLM 에게 파이썬 코드를 작성하도록 요청하고 결과를 반환하는 예제입니다.

흐름 정리
1. LLM 모델에게 특정 작업을 수행하는 Python 코드를 작성하도록 요청합니다.
2. 작성된 코드를 실행하여 결과를 얻습니다.
3. 결과를 출력합니다.


In [7]:
from langchain_community.chat_models import ChatOllama
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnableLambda


# 파이썬 코드를 실행하고 중간 과정을 출력하고 도구 실행 결과를 반환하는 함수
def print_and_execute(code, debug=True):
    if debug:
        print("CODE:")
        print(code)
    return python_tool.invoke(code)


# 파이썬 코드를 작성하도록 요청하는 프롬프트
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are Raymond Hetting, an expert python programmer, well versed in meta-programming and elegant, concise and short but well documented code. You follow the PEP8 style guide. "
            "Return only the code, no intro, no explanation, no chatty, no markdown, no code block, no nothing. Just the code.",
        ),
        ("human", "{input}"),
    ]
)

# Ollama 모델을 불러옵니다.
llm = ChatOllama(model="gemma3:1b")

# 프롬프트와 LLM 모델을 사용하여 체인 생성
chain = prompt | llm | StrOutputParser() | RunnableLambda(print_and_execute)


/var/folders/5d/4yvbq7s16359fdr947szsx5h0000gn/T/ipykernel_15878/3595998609.py:28: LangChainDeprecationWarning: The class `ChatOllama` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import ChatOllama``.
  llm = ChatOllama(model="gemma3:1b")


In [8]:
# 결과 출력
print(chain.invoke("로또 번호 생성기를 출력하는 코드를 작성하세요."))

CODE:
```python
import random

def generate_lottery_numbers(num_numbers=6):
    """
    Generates a list of random lottery numbers.

    Args:
        num_numbers: The number of lottery numbers to generate.  Defaults to 6.

    Returns:
        A list of integers representing the generated lottery numbers.
    """
    numbers = random.sample(range(1, 50), num_numbers)  # Generate unique numbers
    return numbers

if __name__ == "__main__":
    lottery_numbers = generate_lottery_numbers()
    print(lottery_numbers)
```




검색 API 도구

Tavily 검색 API를 활용하여 검색 기능을 구현하는 도구입니다. 이 도구는 두 가지 주요 클래스를 제공합니다: TavilySearchResults와 TavilyAnswer.

In [11]:
from langchain_community.tools.tavily_search import TavilySearchResults

# 도구 생성
tool = TavilySearchResults(
    max_results=6,
    include_answer=True,
    include_raw_content=True,
    # include_images=True,
    # search_depth="advanced", # or "basic"
    include_domains=["github.io", "wikidocs.net"],
    # exclude_domains = []
)

In [12]:
# 도구 실행
tool.invoke({"query": "LangChain Tools 에 대해서 알려주세요"})

[{'url': 'https://jbcodeforce.github.io/ML-studies/coding/langchain/',
  'content': 'LangChain offers an API to the LLM called bind_tools to pass the definition of the tool, as part of each call to the model, so that the application can invoke the tool when appropriate.\n\nSee also the load tools api with a list of predefined tools.\n\nBelow is the classical application flow using tool calling. The exposed function wraps a remote microservice. [...] When developing a solution based on agent, consider the tools, the services, the agent needs to access. See a code example openAI_agent.py.\n\nMany LLM providers support for tool calling, including Anthropic, Cohere, Google, Mistral, OpenAI, see the existing LangChain tools.\n\nInteresting tools¶\n\nSearch recent news¶\n\nA common tool integrated in agent, is the Tavily search API, used to get the last trusted News, so the most recent information created after the cutoff date of the LLM. [...] LangChain Study¶\n\nIn LLM application there ar

In [13]:
from langchain_teddynote.tools.tavily import TavilySearch

# 기본 예제
tavily_tool = TavilySearch()

# include_domains 사용 예제
# 특정 도메인만 포함하여 검색
tavily_tool_with_domains = TavilySearch(include_domains=["github.io", "naver.com"])

# exclude_domains 사용 예제
# 특정 도메인을 제외하고 검색
tavily_tool_exclude = TavilySearch(exclude_domains=["ads.com", "spam.com"])

# 다양한 파라미터를 사용한 검색 예제
result1 = tavily_tool.search(
    query="종각역 맛집 알려줘",  # 검색 쿼리
    search_depth="advanced",  # 고급 검색 수준
    topic="general",  # 일반 주제
    days=7,  # 최근 7일 내 결과
    max_results=10,  # 최대 10개 결과
    include_answer=True,  # 답변 포함
    include_raw_content=True,  # 원본 콘텐츠 포함
    include_images=True,  # 이미지 포함
    format_output=True,  # 결과 포맷팅
)

# 뉴스 검색 예제
result2 = tavily_tool.search(
    query="최신 AI 기술 동향",  # 검색 쿼리
    search_depth="basic",  # 기본 검색 수준
    topic="news",  # 뉴스 주제
    days=3,  # 최근 3일 내 결과
    max_results=5,  # 최대 5개 결과
    include_answer=False,  # 답변 미포함
    include_raw_content=False,  # 원본 콘텐츠 미포함
    include_images=False,  # 이미지 미포함
    format_output=True,  # 결과 포맷팅
)

# 특정 도메인 포함 검색 예제
result3 = tavily_tool_with_domains.search(
    query="파이썬 프로그래밍 팁",  # 검색 쿼리
    search_depth="advanced",  # 고급 검색 수준
    max_results=3,  # 최대 3개 결과
)

# 특정 도메인 제외 검색 예제
result4 = tavily_tool_exclude.search(
    query="건강한 식단",  # 검색 쿼리
    search_depth="basic",  # 기본 검색 수준
    days=30,  # 최근 30일 내 결과
    max_results=7,  # 최대 7개 결과
)

# 결과 출력
print("기본 검색 결과:", result1)
print("뉴스 검색 결과:", result2)
print("특정 도메인 포함 검색 결과:", result3)
print("특정 도메인 제외 검색 결과:", result4)


기본 검색 결과: ["<document><title>종각역 맛집 베스트 추천 top 10 - 코니코니 해피라이프</title><url>https://hotel-iu.tistory.com/4463</url><content>종각역 맛집 베스트 추천 top 10종각역 맛집 베스트 추천 top 10곳을 소개합니다. 1: 선비옥상호명: 선비옥주소: 서울특별시 종로구 관수동 160-1 지상1층(관수동)전화번호: 02-764-2214관련 키워드: ['평양물냉면', '함흥비빔냉면', '한우육회', '종로3가돼지갈비', '종로3가한우']관련 태그: ['데이트하기 좋은</content></document>", "<document><title>종각역 맛집 현지인 추천 베스트 10 (2024)</title><url>https://cncjsgody.mannerguard.com/entry/종각역-맛집-현지인-추천-베스트-10-2024</url><content>오늘의 추천\\n\\n종각역 맛집 현지인 추천 베스트 10 (2024)\\n\\n\\n\\n종각역 여행 중이라면 꼭 가봐야 할 현지인 추천 맛집을 소개합니다. 이번 포스팅에서는 종각역 현지인들이 자주 찾는 인기 맛집 10곳을 소개합니다. 종각역 특유의 맛과 분위기를 즐길 수 있는 다양한 음식점들을 만나보세요. 종각역에서만 맛볼 수 있는 특별한 메뉴를 통해 종각역 여행을 더욱 풍성하게 만들어보세요.\\n\\n\\n\\n\\n\\n\\n\\n1. 양연화로 종로직영점\\n\\n\\n\\n양연화로 종로직영점은 종각역 근처에서 맛있는 돼지고기구이를 즐길 수 있는 고기 전문점으로, 특히 삼겹살의 품질이 뛰어납니다. 신선한 재료와 깔끔한 화로구이 방식은 고기의 풍미를 한층 더 살려줍니다. 넓은 공간과 친절한 서비스 덕분에 가족 모임이나 친구들과의 식사 장소로도 인기가 많습니다. 종각고기집 중에서도 직영점의 신뢰와 맛으로 많은 사랑을 받고 있습니다.\\n\\n\\n\\n👉\xa0양연화로 예약하러 바로가기\\n\\n\\n\\n\\n\\n\\n\\

사용자 정의 도구(Custom Tool)

LangChain 에서 제공하는 빌트인 도구 외에도 사용자가 직접 도구를 정의하여 사용할 수 있습니다.

이를 위해서는 langchain.tools 모듈에서 제공하는 tool 데코레이터를 사용하여 함수를 도구로 변환합니다.

@tool 데코레이터
- 이 데코레이터는 함수를 도구로 변환하는 기능을 제공합니다. 다양한 옵션을 통해 도구의 동작을 커스터마이즈할 수 있습니다.

In [14]:
from langchain.tools import tool


# 데코레이터를 사용하여 함수를 도구로 변환합니다.
@tool
def add_numbers(a: int, b: int) -> int:
    """Add two numbers"""
    return a + b


@tool
def multiply_numbers(a: int, b: int) -> int:
    """Multiply two numbers"""
    return a * b


In [15]:
# 도구 실행
add_numbers.invoke({"a": 3, "b": 4})

7

In [16]:
# 도구 실행
multiply_numbers.invoke({"a": 3, "b": 4})

12

구글 뉴스기사 검색 도구

In [17]:
!pip install -qU langchain-teddynote

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
grpcio-tools 1.62.3 requires protobuf<5.0dev,>=4.21.6, but you have protobuf 5.29.4 which is incompatible.
opentelemetry-proto 1.27.0 requires protobuf<5.0,>=3.19, but you have protobuf 5.29.4 which is incompatible.
google-ai-generativelanguage 0.6.6 requires protobuf!=3.20.0,!=3.20.1,!=4.21.0,!=4.21.1,!=4.21.2,!=4.21.3,!=4.21.4,!=4.21.5,<5.0.0dev,>=3.19.5, but you have protobuf 5.29.4 which is incompatible.
open-webui 0.5.20 requires googleapis-common-protos==1.63.2, but you have googleapis-common-protos 1.69.2 which is incompatible.
open-webui 0.5.20 requires langchain==0.3.19, but you have langchain 0.3.21 which is incompatible.

[notice] A new release of pip is available: 24.0 -> 25.0.1
[notice] To update, run: pip install --upgrade pip


In [18]:
from langchain_teddynote.tools import GoogleNews

# 도구 생성
news_tool = GoogleNews()

In [19]:
# 최신 뉴스 검색
news_tool.search_latest(k=5)

[{'url': 'https://news.google.com/rss/articles/CBMiakFVX3lxTE9CT19TWXNUcGlZZGQxekZlVGczUlJQdWJaSl9iR0VyX3h6a1d4NHAzRmZBWXI5dXJCNUZpWHdUNFBfaGl3U1ViZzgtWG5MUFl4bkhNRFVRSm9YUzNwdkowNk9Fa2RsS3NEQUE?oc=5',
  'content': '산불 결국 지리산까지…사상자 52명 역대 최다 - 한겨레'},
 {'url': 'https://news.google.com/rss/articles/CBMiXEFVX3lxTE8zWUhrQW42SkU0bG5mWEhLU3l5aUFXTUVjNEp0c3U4UHFCMVZfT3RiOWhoYmtmdERvMGNWeVY4aDRqSzN4YlE2aWdPSGd1ZnFXcmZRY0JONWhUYUNP0gFiQVVfeXFMTlJYcXRNckRBVTZ5OGt5dTlwRkk3LU92Z1dQel9xaGVQcTEzVFNiSzdwRFdZd050Y04tbWVBVC1Xak9YSklvUXlqZk4xMGs5UWlBV0I4ZTNPenhZdEo4aTZnRVE?oc=5',
  'content': "'집이 하루아침에 없어졌어요'... 참담한 의성 산불 피해 지역 - BBC.com"},
 {'url': 'https://news.google.com/rss/articles/CBMickFVX3lxTE5MckNSTTVzM1VGZ2R4T05vanFhd3RqY1p1ZUpxLWFBTk5WY1hrMG1yQnByMzV4MjZLLVBQYXVnNkpYMzl1WkM2WHZXYURjRGZRXzhnQy1QUGs4bDNXdGpybU54NXhQYmhPQUlnQks2Yl8zUQ?oc=5',
  'content': '이재명 항소심 재판부 ‘표현의 자유’ 방점…허위 여부 엄격 해석 - 한겨레'},
 {'url': 'https://news.google.com/rss/articles/CBMickFVX3lxTE9iRjJUcjJtRENOR050MnNKRzJSdnl4NDdZM

In [20]:
# 키워드로 뉴스 검색
news_tool.search_by_keyword("AI 투자", k=5)

[{'url': 'https://news.google.com/rss/articles/CBMickFVX3lxTE5TWEZkQlktQVMwMVlCVlQwTjJCS0Y4aTdMS0hnaFdvMGpzUnZnd2d4VzdvSnZQOEtPMlVyU2hiOXJMODVURkppU1h3Z0UxYkF5eEhjUnFfSW5PUjNNS2t1UzhhMnpzODFKem5Wa1hvTlpPdw?oc=5',
  'content': '퓨리오사AI, 메타 인수 거절…벤처투자 생태계 한계 드러나나 - 마켓인'},
 {'url': 'https://news.google.com/rss/articles/CBMiU0FVX3lxTE1TU29IbjUtZ1VBM3E3ZjEyODZYNm04R2I1aVJoOGsxZHktUElOYU1XWHJ5bVA0VWI0ZWVlVnRxMlVHOGNDaFJsSHFtX00takh6R1NN?oc=5',
  'content': '퓨리오사AI, 메타 인수 제안 거절…"투자 유치해 독자적 AI 칩 개발" - 네이트 뉴스'},
 {'url': 'https://news.google.com/rss/articles/CBMiWkFVX3lxTE9Tc1VEa3hUQnliWUZGS0dLX2hGV3RJSTlQZDFCVDVQQ3RDYlZCcDR1WjYwdXFZbXVmemhFX2MyT2VIdUN3eXBoWVV6TkxzelhISlB1cmstc0MxZ9IBVEFVX3lxTFBCQ2xxUDhoUkNMc1FLZkpFVUVaQUNkaHhTZl9ZWUpPb3FQTVhKTWVwZHgyRThOSGoyRlVRYUdhZW5NQkttWWMtSVlXdEZURHI3dklqbw?oc=5',
  'content': '퓨리오사AI, 메타에 안 판다…"투자 유치해 독자 AI칩 개발" - 한국경제'},
 {'url': 'https://news.google.com/rss/articles/CBMiWEFVX3lxTE1VazM2LTZlYzAwTjNNOFVpSFFoQ1IwUGZ4ZVppcmpoMWdjWTdVNTRjSnVFaE1wT0N4Z3JtS05r

In [21]:
from langchain_teddynote.tools import GoogleNews
from langchain.tools import tool
from typing import List, Dict


# 키워드로 뉴스 검색하는 도구 생성
@tool
def search_keyword(query: str) -> List[Dict[str, str]]:
    """Look up news by keyword"""
    print(query)
    news_tool = GoogleNews()
    return news_tool.search_by_keyword(query, k=5)


In [23]:
# 실행 결과
search_keyword.invoke({"query": "개발자"})

개발자


[{'url': 'https://news.google.com/rss/articles/CBMiT0FVX3lxTFBVd1RFd3ptM2doZ3FCTTRtM1lqenNCb3pwaWY0TkZBdl9RMGlwV2hVNExRR0pibVVnZHdtQ2FmcURXY280YmtBMjJEaEd5Tms?oc=5',
  'content': '초급 개발자는 AI 코딩을 좋아하고, 중견 개발자는 싫어한다? - 바이라인네트워크'},
 {'url': 'https://news.google.com/rss/articles/CBMiU0FVX3lxTFAwYjhoNlFXVW5JSzh2VmZjOTVxNVJuQmtQM0U4clVVX3B3Mnhxa2szMVJmQ1NkVzN6ZGdrV1l2Sk9Cc2I1UV84OTVXOXJoUzJJVkVV?oc=5',
  'content': 'AI 활용한 바이브 코딩 시대…개발자 생존전략 3가지 - 네이트 뉴스'},
 {'url': 'https://news.google.com/rss/articles/CBMiU0FVX3lxTE1zbmRLMmROSjBwaWJzdkV5TGwyY2czOWVMYWJ1eU05Zl9ZM0p5UUdXN1pRWloydzZDY1F1RDZ3aFYtekU2QWIyZF9hbUxTWjZrd1VN?oc=5',
  'content': '애플, 6월 9∼13일 세계개발자회의…"운영체제 업데이트 소개" - 네이트 뉴스'},
 {'url': 'https://news.google.com/rss/articles/CBMiTkFVX3lxTE42WktaMFd0a0QwMHluM2pCSVVydTBxNHZibjdaVGNKZnhvbUpWWFBEaUk5VmpsTTNuNmREQ29MWk1PWElROGpvVzdkWGpBZw?oc=5',
  'content': "비아이매트릭스, 'AI 시대 개발자의 길을 찾다' 행사 성료 - 전자신문"},
 {'url': 'https://news.google.com/rss/articles/CBMiakFVX3lxTE5Zak9QcFRNS3BGcGZBNXpZdDQwa